### 1. Load Libraries and Pre-trained Model

In [138]:
import torch
import warnings
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel

warnings.filterwarnings('ignore')
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
else:
    print("cpu")

print("Loading zero-shot classification model...")
print("Model: typeform/distilbert-base-uncased-mnli")

classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",
    device=0 if torch.cuda.is_available() else -1  
)

GPU Device: NVIDIA GeForce RTX 3060
Loading zero-shot classification model...
Model: typeform/distilbert-base-uncased-mnli


Device set to use cuda:0


### 2. Understanding Tokenization & Embeddings

In [139]:
tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")
model = AutoModel.from_pretrained("typeform/distilbert-base-uncased-mnli")
sample_text = "The sky is blue and the grass is green."
tokens = tokenizer(sample_text, return_tensors="pt")
token_strings = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][:12])

with torch.no_grad():
    embeddings = model(**tokens).last_hidden_state

print(f"\n Embedding Tensor Shape: {embeddings.shape}")
print(f"   - Batch size: {embeddings.shape[0]}")
print(f"   - Sequence length: {embeddings.shape[1]} tokens")
print(f"   - Hidden size: {embeddings.shape[2]} dimensions per token")




 Embedding Tensor Shape: torch.Size([1, 12, 768])
   - Batch size: 1
   - Sequence length: 12 tokens
   - Hidden size: 768 dimensions per token


### 3. Define Candidate Color Labels

In [140]:
COLOR_LABELS = [
    "red", "blue", "green", "yellow", "orange", "purple",
    "pink", "brown", "black", "white", "gray", "grey",
    "violet", "indigo", "turquoise", "cyan", "magenta",
    "gold", "silver", "beige", "navy", "maroon",
    "olive", "lime", "teal", "aqua", "coral",
    "crimson", "scarlet", "emerald", "azure", "amber", "ivory"
]
print(f"   Total colors defined: {len(COLOR_LABELS)}")

   Total colors defined: 33


### 4. Core Color Detection Function

In [141]:
def detect_colors(text, confidence_threshold=0.5):
    result = classifier(
        text,
        candidate_labels=COLOR_LABELS,
        multi_label=True  
    )
    detected = [
        (label, score) 
        for label, score in zip(result['labels'], result['scores'])
        if score >= confidence_threshold
    ]
    detected.sort(key=lambda x: x[1], reverse=True)
    return detected

### 5. Baseline Comparison: Semantic vs Literal Detection

In [142]:
def literal_color_detection(text, color_labels=COLOR_LABELS):
    text_lower = text.lower()
    found_colors = []
    
    for color in color_labels:
        if color in text_lower:
            found_colors.append(color)
    return found_colors

comparison_tests = [
    "The sky is blue and grass is green.",
    "I'm feeling blue today.",
    "She wore crimson lipstick.",
    "The painting had azure tones."
]

for text in comparison_tests:
    zero_shot = detect_colors(text, confidence_threshold=0.5)
    literal = literal_color_detection(text)
    
    print(f" Text: '{text}'")
    print(f"    Zero-Shot: {[c for c, _ in zero_shot]}")
    print(f"    Literal:   {literal}")
    
    if zero_shot and not literal:
        print(f"Zero-shot detected semantic/contextual colors!")
    elif literal and not zero_shot:
        print(f"Literal found exact matches missed by model")


 Text: 'The sky is blue and grass is green.'
    Zero-Shot: ['blue', 'green']
    Literal:   ['blue', 'green']
 Text: 'I'm feeling blue today.'
    Zero-Shot: ['blue', 'turquoise', 'indigo', 'navy']
    Literal:   ['blue']
 Text: 'She wore crimson lipstick.'
    Zero-Shot: ['scarlet', 'red', 'crimson', 'violet']
    Literal:   ['crimson']
 Text: 'The painting had azure tones.'
    Zero-Shot: ['azure', 'cyan', 'turquoise', 'magenta', 'indigo']
    Literal:   ['azure']


### 6. Results Display Function

In [143]:
def display_results(text, detected_colors, threshold):
    print(f" TEXT: '{text}'")
    print(f"  Confidence Threshold: {threshold}")
    print(f" Detected {len(detected_colors)} color(s)")
    
    if detected_colors:
        for i, (color, confidence) in enumerate(detected_colors, 1):
            bar_length = int(confidence * 40)
            bar = "█" * bar_length + "░" * (40 - bar_length)
    
            if confidence >= 0.9:
                interpretation = "Very High"
            elif confidence >= 0.7:
                interpretation = "High"
            elif confidence >= 0.5:
                interpretation = "Moderate"
            else:
                interpretation = "Low"
            
            print(f"{i}. {color.upper():<15} | {confidence:.4f} ({interpretation:>10}) | {bar}")
    else:
        print(" No colors detected above threshold.")
        print(" Try lowering the threshold or using different text.")
    

### 7. Threshold Analysis

In [144]:
print("THRESHOLD SENSITIVITY ANALYSIS")
threshold_test_text = "The sky is blue and the grass is green with yellow flowers."
thresholds = [0.3, 0.5, 0.7, 0.9]

print(f"\n Test Text: '{threshold_test_text}'")
print(f" Expected Colors: blue, green, yellow\n")

for t in thresholds:
    detected = detect_colors(threshold_test_text, confidence_threshold=t)
    detected_names = [c for c, _ in detected]
    
    print(f"Threshold {t}:")
    print(f"  ├─ Colors found: {len(detected)}")
    print(f"  ├─ Colors: {detected_names}")
    
    if len(detected) >= 3:
        print(f"  └─  All expected colors detected")
    elif len(detected) >= 2:
        print(f"  └─   Partial detection")
    else:
        print(f"  └─  Missing colors (threshold too high)")
    print()

THRESHOLD SENSITIVITY ANALYSIS

 Test Text: 'The sky is blue and the grass is green with yellow flowers.'
 Expected Colors: blue, green, yellow

Threshold 0.3:
  ├─ Colors found: 3
  ├─ Colors: ['blue', 'yellow', 'green']
  └─  All expected colors detected

Threshold 0.5:
  ├─ Colors found: 2
  ├─ Colors: ['blue', 'yellow']
  └─   Partial detection

Threshold 0.7:
  ├─ Colors found: 2
  ├─ Colors: ['blue', 'yellow']
  └─   Partial detection

Threshold 0.9:
  ├─ Colors found: 2
  ├─ Colors: ['blue', 'yellow']
  └─   Partial detection



In [145]:
print("Testing HARD cases with different thresholds:\n")
test_text = "I'm feeling blue today."
for threshold in [0.5, 0.6, 0.7, 0.8]:
    detected = detect_colors(test_text, threshold)
    print(f"Threshold {threshold}: {[c for c, _ in detected]}")

Testing HARD cases with different thresholds:

Threshold 0.5: ['blue', 'turquoise', 'indigo', 'navy']
Threshold 0.6: ['blue', 'turquoise', 'indigo']
Threshold 0.7: ['blue', 'turquoise', 'indigo']
Threshold 0.8: ['blue', 'turquoise']


### 8. Test Examples

In [146]:
examples = [
    "The sky is blue and the grass is green.",
    "She wore a red dress with yellow flowers and carried a purple purse.",
]

for i, text in enumerate(examples, 1):
    print(f"\n[Example {i}]")
    detected = detect_colors(text, confidence_threshold=0.5)
    display_results(text, detected, threshold=0.5)


[Example 1]
 TEXT: 'The sky is blue and the grass is green.'
  Confidence Threshold: 0.5
 Detected 2 color(s)
1. BLUE            | 0.9979 ( Very High) | ███████████████████████████████████████░
2. GREEN           | 0.8567 (      High) | ██████████████████████████████████░░░░░░

[Example 2]
 TEXT: 'She wore a red dress with yellow flowers and carried a purple purse.'
  Confidence Threshold: 0.5
 Detected 2 color(s)
1. RED             | 0.8595 (      High) | ██████████████████████████████████░░░░░░
2. YELLOW          | 0.8491 (      High) | █████████████████████████████████░░░░░░░
